In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
# from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
def save_obj(obj, name):
    with open('output_kflod/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('output_kflod/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [2]:
def save_obj(obj, name):
    np.save('output_kflod/'+ name + '.npy', dict(tmp_dict))

def load_obj(name):
    return np.load('output_kflod/'+ name + '.npy').flatten()[0]

In [3]:
df = pd.read_csv('df.csv')
user = pd.read_csv('userFeature.csv')

df.shape
df.head()
user.shape
user.head()
# (11064803, 5)

(11064803, 4)

,aid,uid,label,is_his
0,699,78508957,-1,0.0
1,1991,3637295,-1,0.0
2,1119,19229018,-1,0.0
3,2013,79277120,-1,0.0
4,692,41528441,-1,0.0


(11420039, 24)

,uid,age,gender,marriageStatus,education,consumptionAbility,LBS,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,topic1,topic2,topic3,appIdInstall,appIdAction,ct,os,carrier,house
0,26325489,4,2,11,7,2,950.0,93;70;77;86;109;47;75;69;45;8;29;49;83;6;46;36...,46;19;13;29,NaN,NaN,52;100;72;131;116;11;71;12;8;113;28;73;6;132;9...,664359;276966;734911;103617;562294,11395;79112;115065;77033;36176,NaN,9826;105;8525;5488;7281,9708;5553;6745;7477;7150,NaN,NaN,NaN,3;1,2,1,NaN
1,1184123,2,1,5;13,2,1,803.0,75;29,33,NaN,NaN,NaN,338851;361151;542834;496283;229952,80263;39618;53539;180;38163,NaN,4391;9140;5669;1348;4388,9401;7724;1380;8890;7153,NaN,NaN,NaN,3;1,1,1,NaN
2,76072711,1,1,13;10,5,1,927.0,70;12;28;106;59;49;41;6;42;115;35;116;36;11;96,51;22;79;81;70;6;21;4;41;35,NaN,NaN,77;72;80;116;101;13;1;109;8;50;6;42;76;9;46;36...,746140;695808;126355;771775;411511,105115;71378;41409;74061;44005,NaN,1502;5488;9826;2187;8088,5005;9154;2756;5612;4209,NaN,NaN,NaN,3;1,1,3,NaN
3,63071413,4,1,11,7,1,486.0,93;70;77;47;76;75;69;29;59;49;122;42;46;23;17;...,24;70;19;13;30;10,NaN,NaN,100;80;92;37;116;13;47;4;71;8;50;28;98;115;6;4...,283399;402245;734509;654027;32061,74010;32918;67882;116802;20957,NaN,1619;7342;3064;9213;8525,810;2438;5659;1844;9262,NaN,NaN,NaN,1;3,1;2,3,NaN
4,81294159,5,1,11,6,2,112.0,75;29;49;122;36;11,NaN,NaN,NaN,131;116;13;8;6;132;42;9;59;18;58;64;129;103,562294;157603;589706;657719;495672,62764;97803;89066;55545;74061,NaN,477;9826;5808;644;2747,5483;2199;5424;1511;7751,NaN,NaN,NaN,3;1;4;2,1,3,NaN


In [4]:
train = df[df.label != -99]

train.shape

(8798814, 4)

In [5]:
kflod = pd.read_csv('kflod5_auid.csv')

kflod.shape
kflod.groupby(['kflod']).aid.count()

(8798814, 3)

kflod
0    1759904
1    1759835
2    1759756
3    1759691
4    1759628
Name: aid, dtype: int64

In [2]:
split_col = ['marriageStatus', 'interest1', 'interest2', 'interest3', 'interest4', 'interest5', 'kw1', 'kw2', 'kw3',
            'topic1', 'topic2', 'topic3', 'appIdInstall', 'appIdAction', 'ct', 'os']
feature_col = ['age', 'gender', 'marriageStatus', 'education', 'consumptionAbility', 'LBS', 'interest1', 'interest2',
               'interest3', 'interest4', 'interest5', 'kw1', 'kw2', 'kw3', 'topic1', 'topic2', 'topic3', 'appIdInstall',
               'appIdAction', 'ct', 'os', 'carrier']
single_col = ['age', 'gender', 'education', 'consumptionAbility', 'LBS', 'carrier','house']

def spl_col(x):
    if (type(x) == float) | (type(x) == int):
        return x
    elif type(x) == list:
        return x[0].split(';')
    else:
        return x.split(';')

# history

In [7]:
from collections import defaultdict, Counter
import gc
import ast

In [12]:
for k in range(0, 5):
    tmp_his = kflod[kflod.kflod == k]
    tmp_his = tmp_his.merge(train, on=['aid', 'uid'], how='left')
    tmp_his = tmp_his.merge(user, on=['uid'], how='left')
    
    for i in split_col:
        tmp_his[i] = tmp_his[i].apply(spl_col)

    tmp_his_pos = tmp_his[tmp_his.label == 1]
    
    for i in split_col:
        tmp_dict = defaultdict(lambda: 0)
        for j in range(0, tmp_his.shape[0]):
            if type(tmp_his[i][j]) == list:
                for word in tmp_his[i][j]:
                    tmp_dict[word] += 1
        save_obj(tmp_dict, str(i + '_dict_' + str(k)))
        del tmp_dict
        gc.collect()
        
    for i in split_col:
        tmp_dict = defaultdict(lambda: 0)
        for j in list(tmp_his_pos.index):
            if type(tmp_his_pos[i][j]) == list:
                for word in tmp_his_pos[i][j]:
                    tmp_dict[word] += 1
        save_obj(tmp_dict, str(i + '_pos_dict_' + str(k)))
        del tmp_dict
        gc.collect()
        
    for i in feature_col:
        tmp_dict = defaultdict(lambda: 0)
        for j in range(0, tmp_his.shape[0]):
            if type(tmp_his[i][j]) == list:
                tmp_dict[tuple(tmp_his[i][j])] += 1
            elif (type(tmp_his[i][j]) == np.int64) | (type(tmp_his[i][j]) == np.float64):
                tmp_dict[tmp_his[i][j]] += 1
            else:
                tmp_dict[tuple([999])] += 1
        save_obj(tmp_dict, str(i + '_full_dict_' + str(k)))
        del tmp_dict
        gc.collect()
        
    for i in feature_col:
        tmp_dict = defaultdict(lambda: 0)
        for j in list(tmp_his_pos.index):
            if type(tmp_his_pos[i][j]) == list:
                tmp_dict[tuple(tmp_his_pos[i][j])] += 1
            elif (type(tmp_his_pos[i][j]) == np.int64) | (type(tmp_his_pos[i][j]) == np.float64):
                tmp_dict[tmp_his_pos[i][j]] += 1
            else:
                tmp_dict[tuple([999])] += 1
        save_obj(tmp_dict, str(i + '_full_pos_dict_' + str(k)))
        del tmp_dict
        gc.collect()
    
    del tmp_his
    del tmp_his_pos
    gc.collect()

7

0

0

0

0

...

0

0

0

0

21

# train

In [10]:
df.groupby(['label']).aid.count()

train = df[df.label != -99]
test = df[df.label == -99]

label
-99    2265989
-1     8376853
 1      421961
Name: aid, dtype: int64

In [11]:
for k in range(0, 5):
#     tmp_tr = kflod[kflod.kflod == k]
    tmp_tr = test
    tmp_his = kflod[kflod.kflod != k]
    
#     tmp_tr = tmp_tr.merge(train, on=['aid', 'uid'], how='left')
    tmp_tr = tmp_tr.merge(user, on=['uid'], how='left')
    tmp_tr.LBS = tmp_tr.LBS.fillna(tmp_tr.LBS.mean())
    tmp_tr.house = tmp_tr.house.fillna(0)
    
    tmp_his = tmp_his.merge(train, on=['aid', 'uid'], how='left')
    tmp_his = tmp_his.merge(user[single_col + ['uid']], on=['uid'], how='left')
    tmp_his.LBS = tmp_his.LBS.fillna(tmp_his.LBS.mean())
    tmp_his.house = tmp_his.house.fillna(0)
    tmp_his_pos = tmp_his[tmp_his.label == 1]
    
    for i in split_col:
        tmp_tr[i] = tmp_tr[i].apply(spl_col)
    
    li = list(range(0 ,5))
    li.remove(k)
    
    for i in split_col:
        for ti in li:
            locals()['tmp_dict' + str(li.index(ti) + 1)] = load_obj(str(i + '_dict_' + str(ti)))
        tmp_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4)
        tmp_dict = dict(tmp_dict)
        
        for ti in li:
            locals()['tmp_dict' + str(li.index(ti) + 1)] = load_obj(str(i + '_pos_dict_' + str(ti)))
        tmp_pos_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4)
        tmp_pos_dict = dict(tmp_pos_dict)
        
        del tmp_dict1
        del tmp_dict2
        del tmp_dict3
        del tmp_dict4
        gc.collect()
        
        tmp_list1 = list(np.zeros(tmp_tr.shape[0]))
        tmp_list2 = list(np.zeros(tmp_tr.shape[0]))
        for kk in range(0, tmp_tr.shape[0]):
            if type(tmp_tr[i][kk]) == list:
                for j in tmp_tr[i][kk]:
                    if j in tmp_dict.keys():
                        tmp_list1[kk] = tmp_list1[kk] + tmp_dict[j]
                    if j in tmp_pos_dict.keys():
                        tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[j]
        tmp_tr[str(i + '_split_count_score')] = tmp_list1
        tmp_tr[str(i + '_split_pos_count_score')] = tmp_list2
        
        del tmp_dict
        del tmp_pos_dict
        del tmp_list1
        del tmp_list2
        gc.collect()
        
    for i in feature_col:
        for ti in li:
            locals()['tmp_dict' + str(li.index(ti) + 1)] = load_obj(str(i + '_full_dict_' + str(ti)))
        tmp_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4)
        tmp_dict = dict(tmp_dict)
        
        for ti in li:
            locals()['tmp_dict' + str(li.index(ti) + 1)] = load_obj(str(i + '_full_pos_dict_' + str(ti)))
        tmp_pos_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4)
        tmp_pos_dict = dict(tmp_pos_dict)
        
        del tmp_dict1
        del tmp_dict2
        del tmp_dict3
        del tmp_dict4
        gc.collect()
        
        tmp_list1 = list(np.zeros(tmp_tr.shape[0]))
        tmp_list2 = list(np.zeros(tmp_tr.shape[0]))
        for kk in range(0, tmp_tr.shape[0]):
            if type(tmp_tr[i][kk]) == list:
                if tuple(tmp_tr[i][kk]) in tmp_dict.keys():
                    tmp_list1[kk] = tmp_list1[kk] + tmp_dict[tuple(tmp_tr[i][kk])]
                if tuple(tmp_tr[i][kk]) in tmp_pos_dict.keys():
                    tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[tuple(tmp_tr[i][kk])]
            elif (type(tmp_tr[i][kk]) == np.int64) | (type(tmp_tr[i][kk]) == np.float64):
                if tmp_tr[i][kk] in tmp_dict.keys():
                    tmp_list1[kk] = tmp_list1[kk] + tmp_dict[tmp_tr[i][kk]]
                if tmp_tr[i][kk] in tmp_pos_dict.keys():
                    tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[tmp_tr[i][kk]]
        tmp_tr[str(i + '_feature_count_score')] = tmp_list1
        tmp_tr[str(i + '_feature_pos_count_score')] = tmp_list2
        
        del tmp_dict
        del tmp_pos_dict
        del tmp_list1
        del tmp_list2
        gc.collect()
        
    for i in single_col:
        locals()[str(i + '_feature')] = tmp_his.groupby(
            ['aid'], as_index=False)[i].agg({
                str(i + '_max'):
                'max',
                str(i + '_min'):
                'min',
                str(i + '_mean'):
                'mean',
                str(i + '_std'):
                'std'
            })
        locals()[str(i + '_pos_feature')] = tmp_his_pos.groupby(
            ['aid'], as_index=False)[i].agg({
                str(i + '_pos_max'):
                'max',
                str(i + '_pos_min'):
                'min',
                str(i + '_pos_mean'):
                'mean',
                str(i + '_pos_std'):
                'std'
            })
        tmp_tr = tmp_tr.merge(locals()[str(i + '_feature')], on=['aid'], how='left')
        tmp_tr = tmp_tr.merge(locals()[str(i + '_pos_feature')], on=['aid'], how='left')
        tmp_tr[str(i + '_mean_sub')] = abs(tmp_tr[i] - tmp_tr[str(i + '_mean')])
        tmp_tr[str(i + '_mean_pos_sub')] = abs(tmp_tr[i] - tmp_tr[str(i + '_pos_mean')])
        del locals()[str(i + '_feature')]
        del locals()[str(i + '_pos_feature')]
        gc.collect()
        
    tmp_tr.to_csv('test_kflod' + str(k), index=None)
    del tmp_tr
    del tmp_his
    del tmp_his_pos
    gc.collect()

24

28

24

14

24

...

70

70

70

70

7

# test

In [24]:
test = df[df.label == -99]
test.reset_index(drop=True, inplace=True)

test.shape
test.head()

(2265989, 4)

,aid,uid,label,is_his
0,2118,64355836,-99,-99.0
1,692,45051997,-99,-99.0
2,692,10869198,-99,-99.0
3,1918,75929554,-99,-99.0
4,1596,5790162,-99,-99.0


In [31]:
test = df[df.label == -99]
test.reset_index(drop=True, inplace=True)
test = test.merge(user, on=['uid'], how='left')
test.LBS = test.LBS.fillna(test.LBS.mean())
test.house = test.house.fillna(0)
    
for i in split_col:
    test[i] = test[i].apply(spl_col)

tmp_his = df[df.label != -99]
tmp_his = tmp_his.merge(user[single_col + ['uid']], on=['uid'], how='left')
tmp_his.LBS = tmp_his.LBS.fillna(tmp_his.LBS.mean())
tmp_his.house = tmp_his.house.fillna(0)
tmp_his_pos = tmp_his[tmp_his.label == 1]

for i in split_col:
    for ti in range(0, 5):
        locals()['tmp_dict' + str(ti + 1)] = load_obj(str(i + '_dict_' + str(ti)))
    tmp_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4) + Counter(tmp_dict5)
    tmp_dict = dict(tmp_dict)
    
    for ti in range(0, 5):
        locals()['tmp_dict' + str(ti + 1)] = load_obj(str(i + '_pos_dict_' + str(ti)))
    tmp_pos_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4) + Counter(tmp_dict5)
    tmp_pos_dict = dict(tmp_pos_dict)
        
    del tmp_dict1
    del tmp_dict2
    del tmp_dict3
    del tmp_dict4
    del tmp_dict5
    gc.collect()
        
    tmp_list1 = list(np.zeros(test.shape[0]))
    tmp_list2 = list(np.zeros(test.shape[0]))
    for kk in range(0, test.shape[0]):
        if type(test[i][kk]) == list:
            for j in test[i][kk]:
                if j in test.keys():
                    tmp_list1[kk] = tmp_list1[kk] + tmp_dict[j]
                if j in tmp_pos_dict.keys():
                    tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[j]
    test[str(i + '_split_count_score')] = tmp_list1
    test[str(i + '_split_pos_count_score')] = tmp_list2
        
    del tmp_dict
    del tmp_pos_dict
    del tmp_list1
    del tmp_list2
    gc.collect()
    
for i in feature_col:
    for ti in range(0, 5):
        locals()['tmp_dict' + str(ti + 1)] = load_obj(str(i + '_full_dict_' + str(ti)))
    tmp_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4) + Counter(tmp_dict5)
    tmp_dict = dict(tmp_dict)
    
    for ti in range(0, 5):
        locals()['tmp_dict' + str(ti + 1)] = load_obj(str(i + '_full_pos_dict_' + str(ti)))
    tmp_pos_dict = Counter(tmp_dict1) + Counter(tmp_dict2) + Counter(tmp_dict3) + Counter(tmp_dict4) + Counter(tmp_dict5)
    tmp_pos_dict = dict(tmp_pos_dict)
        
    del tmp_dict1
    del tmp_dict2
    del tmp_dict3
    del tmp_dict4
    del tmp_dict5
    gc.collect()
        
    tmp_list1 = list(np.zeros(test.shape[0]))
    tmp_list2 = list(np.zeros(test.shape[0]))
    for kk in range(0, test.shape[0]):
        if type(test[i][kk]) == list:
            if tuple(test[i][kk]) in tmp_dict.keys():
                tmp_list1[kk] = tmp_list1[kk] + tmp_dict[tuple(test[i][kk])]
            if tuple(test[i][kk]) in tmp_pos_dict.keys():
                tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[tuple(test[i][kk])]
        elif (type(test[i][kk]) == np.int64) | (type(test[i][kk]) == np.float64):
            if test[i][kk] in tmp_dict.keys():
                tmp_list1[kk] = tmp_list1[kk] + tmp_dict[test[i][kk]]
            if test[i][kk] in tmp_pos_dict.keys():
                tmp_list2[kk] = tmp_list2[kk] + tmp_pos_dict[test[i][kk]]
    test[str(i + '_feature_count_score')] = tmp_list1
    test[str(i + '_feature_pos_count_score')] = tmp_list2
        
    del tmp_dict
    del tmp_pos_dict
    del tmp_list1
    del tmp_list2
    gc.collect()
    
for i in single_col:
    locals()[str(i + '_feature')] = tmp_his.groupby(
        ['aid'], as_index=False)[i].agg({
            str(i + '_max'):
            'max',
            str(i + '_min'):
            'min',
            str(i + '_mean'):
            'mean',
            str(i + '_std'):
            'std'
        })
    locals()[str(i + '_pos_feature')] = tmp_his_pos.groupby(
        ['aid'], as_index=False)[i].agg({
            str(i + '_pos_max'):
            'max',
            str(i + '_pos_min'):
            'min',
            str(i + '_pos_mean'):
            'mean',
            str(i + '_pos_std'):
            'std'
        })
    test = test.merge(locals()[str(i + '_feature')], on=['aid'], how='left')
    test = test.merge(locals()[str(i + '_pos_feature')], on=['aid'], how='left')
    test[str(i + '_mean_sub')] = abs(test[i] - test[str(i + '_mean')])
    test[str(i + '_mean_pos_sub')] = abs(test[i] - test[str(i + '_pos_mean')])
    del locals()[str(i + '_feature')]
    del locals()[str(i + '_pos_feature')]
    gc.collect()
        
test.to_csv('test_kflod', index=None)
del test
del tmp_his
del tmp_his_pos
gc.collect()

4739

14

30

14

30

...

70

70

70

70

7

# xgb分开

In [3]:
import xgboost as xgb
import gc

In [4]:
full_col = ['carrier_pos_max','carrier_min','gender_pos_min','interest1_feature_count_score','carrier','carrier_feature_count_score','kw1_feature_pos_count_score','interest2_feature_count_score','appIdAction_split_pos_count_score','education_max','ct_feature_count_score','interest2_feature_pos_count_score','topic3_feature_count_score','house_pos_mean','LBS_mean_sub','education','kw1_split_pos_count_score','os_split_pos_count_score','appIdInstall_split_pos_count_score','interest4_split_count_score','interest5_split_pos_count_score','LBS_pos_mean','topic2_feature_pos_count_score','education_std','carrier_std','consumptionAbility_pos_std','gender_std','interest2_split_pos_count_score','age_std','kw2_split_pos_count_score','gender_mean_sub','LBS_pos_std','topic3_split_pos_count_score','consumptionAbility_max','carrier_max','LBS_feature_count_score','consumptionAbility_mean','house_pos_max','appIdAction_feature_pos_count_score','LBS_min','consumptionAbility_mean_pos_sub','consumptionAbility_feature_count_score','house_mean','age_mean','house_pos_min','house_pos_std','LBS_mean_pos_sub','interest1_feature_pos_count_score','ct_split_count_score','topic2_split_pos_count_score','topic2_feature_count_score','topic1_split_count_score','age_feature_pos_count_score','aid','interest3_split_count_score','kw3_split_count_score','house_max','interest5_feature_count_score','kw2_feature_count_score','LBS_std','house_mean_sub','LBS_mean','kw3_split_pos_count_score','topic1_split_pos_count_score','interest1_split_pos_count_score','gender','gender_pos_mean','gender_feature_count_score','appIdAction_feature_count_score','carrier_mean_pos_sub','interest3_split_pos_count_score','age_pos_min','education_pos_mean','house_std','age_mean_sub','marriageStatus_feature_count_score','appIdInstall_feature_pos_count_score','house','education_pos_max','consumptionAbility_pos_min','age_feature_count_score','education_mean_pos_sub','interest3_feature_count_score','age_mean_pos_sub','gender_pos_max','uid','kw2_split_count_score','topic2_split_count_score','education_feature_count_score','interest4_feature_pos_count_score','age_max','carrier_pos_min','gender_feature_pos_count_score','consumptionAbility_std','education_pos_min','interest4_split_pos_count_score','consumptionAbility_pos_mean','interest5_split_count_score','os_feature_count_score','os_split_count_score','label','appIdInstall_split_count_score','LBS_pos_max','kw3_feature_count_score','topic3_split_count_score','topic1_feature_pos_count_score','education_pos_std','LBS_pos_min','carrier_pos_mean','topic1_feature_count_score','education_min','carrier_mean','education_mean','kw2_feature_pos_count_score','consumptionAbility_pos_max','consumptionAbility_mean_sub','gender_pos_std','age_min','interest4_feature_count_score','ct_split_pos_count_score','LBS','gender_min','marriageStatus_split_pos_count_score','gender_mean_pos_sub','carrier_feature_pos_count_score','topic3_feature_pos_count_score','interest5_feature_pos_count_score','age_pos_max','appIdAction_split_count_score','LBS_max','kw1_feature_count_score','marriageStatus_split_count_score','education_mean_sub','consumptionAbility_min','marriageStatus_feature_pos_count_score','consumptionAbility_feature_pos_count_score','appIdInstall_feature_count_score','LBS_feature_pos_count_score','interest3_feature_pos_count_score','age','os_feature_pos_count_score','interest1_split_count_score','kw1_split_count_score','kw3_feature_pos_count_score','ct_feature_pos_count_score','age_pos_std','carrier_pos_std','gender_mean','gender_max','age_pos_mean','house_mean_pos_sub','house_min','education_feature_pos_count_score','consumptionAbility','interest2_split_count_score','carrier_mean_sub']

In [ ]:
# full_col = list(set(train1.columns) - set(split_col) - set(['is_his', 'kflod']))
predictors = list(set(train1.columns) - set(split_col) - set(['is_his', 'aid', 'uid', 'label', 'kflod']))
target = ['label']
len(set(predictors) & set(train1.columns))
# len(set(predictors) & set(test.columns))
len(predictors)

In [11]:
#cv
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'auc',
    'seed': 201803,
    'missing': 0
}

len(predictors)

train1['label'][train1.label == -1] = 0
xgbtrain = xgb.DMatrix(train1[predictors], train1[target])

cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=10000,
    nfold=3,
    verbose_eval=100,
    early_stopping_rounds=50,
    seed=201803)
print(str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-auc-mean']))
# 1980:0.72182

1980:0.72182


In [6]:
test = pd.read_csv('test_kflod', usecols=['aid', 'uid'])

test.shape
test.head()

(2265989, 2)

,aid,uid
0,2118,64355836
1,692,45051997
2,692,10869198
3,1918,75929554
4,1596,5790162


In [14]:
#train/test
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
#     'scale_pos_weight': int(train[train.label == 0].shape[0]/train[train.label == 1].shape[0]),
    'verbose_eval': 1,
    'eval_metric': 'auc',
    'seed': 201803,
    'missing': -1
}

# for i in range(0, 5):
#     tmp_train = pd.read_csv('tr_kflod' + str(i), usecols=full_col)
#     tmp_test = pd.read_csv('test_kflod' + str(i), usecols=full_col)
#     predictors = list(set(tmp_train.columns) - set(split_col) - set(['is_his', 'aid', 'uid', 'label', 'kflod']))
#     len(predictors)
#     target = ['label']
#     print('start kflod_' + str(i))
#     tmp_train['label'][tmp_train.label == -1] = 0
#     xgbtrain = xgb.DMatrix(tmp_train[predictors], tmp_train[target])
#     xgbtest = xgb.DMatrix(tmp_test[predictors])

#     model = xgb.train(params, xgbtrain, num_boost_round=1980, verbose_eval=100)
#     y_pred = model.predict(xgbtest)
#     y_pred = pd.Series(y_pred, name='test')
    
#     test['score_' + str(i)] = list(y_pred)
#     print('finish kflod_' + str(i))
    
#     del tmp_train
#     del tmp_test
#     gc.collect()
    
test['score'] = (test['score_0'] + test['score_1'] + test['score_2'] + test['score_3'] + test['score_4']) / 5

In [15]:
test[['aid','uid','score_0','score_1','score_2','score_3','score_4','score']].head()

,aid,uid,score_0,score_1,score_2,score_3,score_4,score
0,2118,64355836,0.046616,0.047110,0.042533,0.043293,0.048661,0.045643
1,692,45051997,0.029959,0.031260,0.037797,0.040466,0.032353,0.034367
2,692,10869198,0.034535,0.040661,0.027282,0.032500,0.028964,0.032788
3,1918,75929554,0.031406,0.038289,0.031384,0.042147,0.035950,0.035835
4,1596,5790162,0.138514,0.214496,0.153955,0.207861,0.194549,0.181875


In [16]:
import os
result = test[['aid','uid','score_1']]
result.columns = ['aid','uid','score']
result.to_csv('submission.csv', index=False, float_format='%.8f')
os.system('zip ./result/xgb_kflod_new.zip submission.csv')

0

# xgb合体

In [3]:
import xgboost as xgb
import gc

In [4]:
full_col = ['carrier_pos_max','carrier_min','gender_pos_min','interest1_feature_count_score','carrier','carrier_feature_count_score','kw1_feature_pos_count_score','interest2_feature_count_score','appIdAction_split_pos_count_score','education_max','ct_feature_count_score','interest2_feature_pos_count_score','topic3_feature_count_score','house_pos_mean','LBS_mean_sub','education','kw1_split_pos_count_score','os_split_pos_count_score','appIdInstall_split_pos_count_score','interest4_split_count_score','interest5_split_pos_count_score','LBS_pos_mean','topic2_feature_pos_count_score','education_std','carrier_std','consumptionAbility_pos_std','gender_std','interest2_split_pos_count_score','age_std','kw2_split_pos_count_score','gender_mean_sub','LBS_pos_std','topic3_split_pos_count_score','consumptionAbility_max','carrier_max','LBS_feature_count_score','consumptionAbility_mean','house_pos_max','appIdAction_feature_pos_count_score','LBS_min','consumptionAbility_mean_pos_sub','consumptionAbility_feature_count_score','house_mean','age_mean','house_pos_min','house_pos_std','LBS_mean_pos_sub','interest1_feature_pos_count_score','ct_split_count_score','topic2_split_pos_count_score','topic2_feature_count_score','topic1_split_count_score','age_feature_pos_count_score','aid','interest3_split_count_score','kw3_split_count_score','house_max','interest5_feature_count_score','kw2_feature_count_score','LBS_std','house_mean_sub','LBS_mean','kw3_split_pos_count_score','topic1_split_pos_count_score','interest1_split_pos_count_score','gender','gender_pos_mean','gender_feature_count_score','appIdAction_feature_count_score','carrier_mean_pos_sub','interest3_split_pos_count_score','age_pos_min','education_pos_mean','house_std','age_mean_sub','marriageStatus_feature_count_score','appIdInstall_feature_pos_count_score','house','education_pos_max','consumptionAbility_pos_min','age_feature_count_score','education_mean_pos_sub','interest3_feature_count_score','age_mean_pos_sub','gender_pos_max','uid','kw2_split_count_score','topic2_split_count_score','education_feature_count_score','interest4_feature_pos_count_score','age_max','carrier_pos_min','gender_feature_pos_count_score','consumptionAbility_std','education_pos_min','interest4_split_pos_count_score','consumptionAbility_pos_mean','interest5_split_count_score','os_feature_count_score','os_split_count_score','label','appIdInstall_split_count_score','LBS_pos_max','kw3_feature_count_score','topic3_split_count_score','topic1_feature_pos_count_score','education_pos_std','LBS_pos_min','carrier_pos_mean','topic1_feature_count_score','education_min','carrier_mean','education_mean','kw2_feature_pos_count_score','consumptionAbility_pos_max','consumptionAbility_mean_sub','gender_pos_std','age_min','interest4_feature_count_score','ct_split_pos_count_score','LBS','gender_min','marriageStatus_split_pos_count_score','gender_mean_pos_sub','carrier_feature_pos_count_score','topic3_feature_pos_count_score','interest5_feature_pos_count_score','age_pos_max','appIdAction_split_count_score','LBS_max','kw1_feature_count_score','marriageStatus_split_count_score','education_mean_sub','consumptionAbility_min','marriageStatus_feature_pos_count_score','consumptionAbility_feature_pos_count_score','appIdInstall_feature_count_score','LBS_feature_pos_count_score','interest3_feature_pos_count_score','age','os_feature_pos_count_score','interest1_split_count_score','kw1_split_count_score','kw3_feature_pos_count_score','ct_feature_pos_count_score','age_pos_std','carrier_pos_std','gender_mean','gender_max','age_pos_mean','house_mean_pos_sub','house_min','education_feature_pos_count_score','consumptionAbility','interest2_split_count_score','carrier_mean_sub']

In [5]:
train1 = pd.read_csv('tr_kflod0', usecols=full_col)
train2 = pd.read_csv('tr_kflod1', usecols=full_col)
train3 = pd.read_csv('tr_kflod2', usecols=full_col)
train4 = pd.read_csv('tr_kflod3', usecols=full_col)
train5 = pd.read_csv('tr_kflod4', usecols=full_col)
# test1 = pd.read_csv('test_kflod0', usecols=full_col)
# test2 = pd.read_csv('test_kflod1', usecols=full_col)
# test3 = pd.read_csv('test_kflod2', usecols=full_col)
# test4 = pd.read_csv('test_kflod3', usecols=full_col)
# test5 = pd.read_csv('test_kflod4', usecols=full_col)


train1.shape
train2.shape
train3.shape
train4.shape
train5.shape
# test1.shape
# test2.shape
# test3.shape
# test4.shape
# test5.shape

(1759904, 156)

(1759835, 156)

(1759756, 156)

(1759691, 156)

(1759628, 156)

In [6]:
train = pd.concat([train1, train2, train3, train4, train5])

del train1
del train2
del train3
del train4
del train5

gc.collect()

28

In [7]:
# full_col = list(set(train1.columns) - set(split_col) - set(['is_his', 'kflod']))
predictors = list(set(train.columns) - set(split_col) - set(['is_his', 'aid', 'uid', 'label', 'kflod']))
target = ['label']
len(set(predictors) & set(train.columns))
# len(set(predictors) & set(test.columns))
len(predictors)

153

153

In [ ]:
#cv
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'auc',
    'seed': 201803,
    'missing': 0
}

len(predictors)

train['label'][train.label == -1] = 0
xgbtrain = xgb.DMatrix(train[predictors], train[target])

cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=10000,
    nfold=3,
    verbose_eval=100,
    early_stopping_rounds=50,
    seed=201803)
print(str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-auc-mean']))

153

In [6]:
test = pd.read_csv('test_kflod', usecols=['aid', 'uid'])

test.shape
test.head()

(2265989, 2)

,aid,uid
0,2118,64355836
1,692,45051997
2,692,10869198
3,1918,75929554
4,1596,5790162


In [ ]:
# 整合跑模型
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
#     'scale_pos_weight': int(train[train.label == 0].shape[0]/train[train.label == 1].shape[0]),
    'verbose_eval': 1,
    'eval_metric': 'auc',
    'seed': 201803,
    'missing': -1
}

tmp_train = pd.read_csv('tr_kflod' + str(i), usecols=full_col)
tmp_test = pd.read_csv('test_kflod' + str(i), usecols=full_col)
predictors = list(set(tmp_train.columns) - set(split_col) - set(['is_his', 'aid', 'uid', 'label', 'kflod']))
len(predictors)
target = ['label']
print('start kflod_' + str(i))
tmp_train['label'][tmp_train.label == -1] = 0
xgbtrain = xgb.DMatrix(tmp_train[predictors], tmp_train[target])
xgbtest = xgb.DMatrix(tmp_test[predictors])

model = xgb.train(params, xgbtrain, num_boost_round=1980, verbose_eval=100)
y_pred = model.predict(xgbtest)
y_pred = pd.Series(y_pred, name='test')
    
test['score_' + str(i)] = list(y_pred)
print('finish kflod_' + str(i))
    
del tmp_train
del tmp_test
gc.collect()
    
test['score'] = (test['score_0'] + test['score_1'] + test['score_2'] + test['score_3'] + test['score_4']) / 5

In [16]:
test.head()

,aid,uid,label,is_his,age,gender,marriageStatus,education,consumptionAbility,LBS,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,topic1,topic2,topic3,appIdInstall,appIdAction,ct,os,carrier,house,marriageStatus_split_count_score,marriageStatus_split_pos_count_score,interest1_split_count_score,interest1_split_pos_count_score,interest2_split_count_score,interest2_split_pos_count_score,interest3_split_count_score,interest3_split_pos_count_score,interest4_split_count_score,interest4_split_pos_count_score,interest5_split_count_score,interest5_split_pos_count_score,kw1_split_count_score,kw1_split_pos_count_score,kw2_split_count_score,kw2_split_pos_count_score,kw3_split_count_score,kw3_split_pos_count_score,topic1_split_count_score,topic1_split_pos_count_score,topic2_split_count_score,topic2_split_pos_count_score,topic3_split_count_score,topic3_split_pos_count_score,appIdInstall_split_count_score,appIdInstall_split_pos_count_score,appIdAction_split_count_score,appIdAction_split_pos_count_score,ct_split_count_score,ct_split_pos_count_score,os_split_count_score,os_split_pos_count_score,age_feature_count_score,age_feature_pos_count_score,gender_feature_count_score,gender_feature_pos_count_score,marriageStatus_feature_count_score,marriageStatus_feature_pos_count_score,education_feature_count_score,education_feature_pos_count_score,consumptionAbility_feature_count_score,consumptionAbility_feature_pos_count_score,LBS_feature_count_score,LBS_feature_pos_count_score,interest1_feature_count_score,interest1_feature_pos_count_score,interest2_feature_count_score,interest2_feature_pos_count_score,interest3_feature_count_score,interest3_feature_pos_count_score,interest4_feature_count_score,interest4_feature_pos_count_score,interest5_feature_count_score,interest5_feature_pos_count_score,kw1_feature_count_score,kw1_feature_pos_count_score,kw2_feature_count_score,kw2_feature_pos_count_score,kw3_feature_count_score,kw3_feature_pos_count_score,topic1_feature_count_score,topic1_feature_pos_count_score,topic2_feature_count_score,topic2_feature_pos_count_score,topic3_feature_count_score,topic3_feature_pos_count_score,appIdInstall_feature_count_score,appIdInstall_feature_pos_count_score,appIdAction_feature_count_score,appIdAction_feature_pos_count_score,ct_feature_count_score,ct_feature_pos_count_score,os_feature_count_score,os_feature_pos_count_score,carrier_feature_count_score,carrier_feature_pos_count_score,age_max,age_min,age_mean,age_std,age_pos_max,age_pos_min,age_pos_mean,age_pos_std,age_mean_sub,age_mean_pos_sub,gender_max,gender_min,gender_mean,gender_std,gender_pos_max,gender_pos_min,gender_pos_mean,gender_pos_std,gender_mean_sub,gender_mean_pos_sub,education_max,education_min,education_mean,education_std,education_pos_max,education_pos_min,education_pos_mean,education_pos_std,education_mean_sub,education_mean_pos_sub,consumptionAbility_max,consumptionAbility_min,consumptionAbility_mean,consumptionAbility_std,consumptionAbility_pos_max,consumptionAbility_pos_min,consumptionAbility_pos_mean,consumptionAbility_pos_std,consumptionAbility_mean_sub,consumptionAbility_mean_pos_sub,LBS_max,LBS_min,LBS_mean,LBS_std,LBS_pos_max,LBS_pos_min,LBS_pos_mean,LBS_pos_std,LBS_mean_sub,LBS_mean_pos_sub,carrier_max,carrier_min,carrier_mean,carrier_std,carrier_pos_max,carrier_pos_min,carrier_pos_mean,carrier_pos_std,carrier_mean_sub,carrier_mean_pos_sub,house_max,house_min,house_mean,house_std,house_pos_max,house_pos_min,house_pos_mean,house_pos_std,house_mean_sub,house_mean_pos_sub,score_1,score_2,score_3,score_4,score_5,score
0,2118,64355836,-99,-99.0,1,1,"['13', '10']",6,2,484.0,"['70', '86', '100', '76', '75', '28', '106', '...","['47', '22', '24', '73', '9', '1', '46', '32',...",NaN,NaN,"['72', '37', '13', '79', '6', '46', '94', '59'...","['89154', '196312', '627369', '87552', '440482']","['119567', '104708', '61089', '4272', '90835']",NaN,"['9163', '1390', '9359', '7441', '2266']","['8582', '1471', '679', '4552', '845']",NaN,NaN,NaN,"['1', '3', '4